In [1]:
# !pip install torchtext==0.12.0

In [2]:
!pip install datasets

In [3]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.nn import functional as F
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from transformers import T5Tokenizer
from datasets import load_dataset
import re
import string
import nltk

In [4]:
# from torchtext.vocab import GloVe

# # Load GloVe embeddings
# glove = GloVe(name='6B', dim=300)  # 50d vectors from GloVe
# vector = glove['king']



In [5]:
dataset = load_dataset("xsum", trust_remote_code=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

README.md:   0%|          | 0.00/6.24k [00:00<?, ?B/s]

xsum.py:   0%|          | 0.00/5.76k [00:00<?, ?B/s]

(…)SUM-EMNLP18-Summary-Data-Original.tar.gz:   0%|          | 0.00/255M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

cuda


In [6]:
traindata = dataset['train']
valdata = dataset['validation']
test_data = dataset['test']
df_train = pd.DataFrame(traindata)
df_val = pd.DataFrame(valdata)
df_test = pd.DataFrame(test_data)
X_traindata, y_traindata = df_train['document'], df_train['summary']
X_valdata, y_valdata = df_val['document'], df_val['summary']
X_testdata, y_testdata = df_test['document'], df_test['summary']


In [7]:
#Pre-processing input
class SummaryDataset(Dataset):
    def __init__(self, articles, highlights, tokenizer, encoder_maxlen, decoder_maxlen):
        self.articles = articles
        self.highlights = highlights
        self.tokenizer = tokenizer
        self.encoder_maxlen = encoder_maxlen
        self.decoder_maxlen = decoder_maxlen

    def preprocess_text(self, text):
        text = text.lower()
        text = re.sub(r'[^a-z0-9\s.,!?%$&@#:]', '', text)
        text = ' '.join(text.split())
        return text

    def __len__(self):
        return len(self.articles)

    def __getitem__(self, idx):
        article = self.preprocess_text(str(self.articles[idx]))
        highlight = self.preprocess_text(str(self.highlights[idx]))

        article_encodings = self.tokenizer(
            article,
            max_length=self.encoder_maxlen,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )

        highlight_encodings = self.tokenizer(
            highlight,
            max_length=self.decoder_maxlen,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )

        return (
            article_encodings['input_ids'].squeeze(0),
            article_encodings['attention_mask'].squeeze(0),
            highlight_encodings['input_ids'].squeeze(0)
        )

tokenizer = T5Tokenizer.from_pretrained('t5-base')
max_length = 512
decoder_maxlen = 50
encoder_maxlen = 512


train_dataset = SummaryDataset(X_traindata, y_traindata, tokenizer, encoder_maxlen, decoder_maxlen)
val_dataset = SummaryDataset(X_valdata, y_valdata, tokenizer, encoder_maxlen, decoder_maxlen)
test_dataset = SummaryDataset(X_testdata, y_testdata, tokenizer, encoder_maxlen, decoder_maxlen)

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [8]:
# def load_glove_embeddings(vocab_size, glove, tokenizer, d_model=300):
#   weights_matrix = np.zeros((vocab_size, d_model))
  
#   # Initialize with random values for tokens not in GloVe
#   weights_matrix = np.random.normal(scale=0.6, size=(vocab_size, d_model))
  
#   # Map tokenizer vocabulary to GloVe embeddings
#   for i in range(vocab_size):
#       try:
#           token = tokenizer.decode([i])
#           if token in glove.stoi:
#               weights_matrix[i] = glove[token]
#       except:
#           continue
          
#   return torch.FloatTensor(weights_matrix)

In [9]:
def angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    return pos * angle_rates
def positional_encoding(position, d_model):
    angle_rads = angles(np.arange(position)[:, np.newaxis],
                       np.arange(d_model)[np.newaxis, :],
                       d_model)
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    pos_encoding = angle_rads[np.newaxis, ...]
    return torch.FloatTensor(pos_encoding).to(device)

In [10]:
def create_padding_mask(seq):
    mask = (seq == 0).float()
    return mask[:, None, None, :]

def create_look_ahead_mask(size):
    mask = torch.triu(torch.ones((size, size)), diagonal=1)
    return mask

In [11]:
def scaled_dot_product_attention(q, k, v, mask=None):
  q, k, v = q.to(device), k.to(device), v.to(device)
  depth = q.size(-1)
  dots = torch.matmul(q, k.transpose(-2, -1))

  if mask is not None:
      dots = dots.masked_fill(mask.bool(), -1e9)
  attention_weights = F.softmax(dots, dim=-1)
  output = torch.matmul(attention_weights, v)

  return output, attention_weights
def pointwise_ffn(d_model, dff):
  return torch.nn.Sequential(
      torch.nn.Linear(d_model, dff),
      torch.nn.ReLU(),
      torch.nn.Linear(dff, d_model)
  ).to(device)

In [12]:
class MultiHeadAttention(nn.Module):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads
    #Linear layer for q k v
    self.wq = nn.Linear(d_model, d_model)
    self.wk = nn.Linear(d_model, d_model)
    self.wv = nn.Linear(d_model, d_model)
    # Final linear layer
    self.dense = nn.Linear(d_model, d_model)
  def split_heads(self, x, batch_size):  # Added split_heads function
        x = x.view(batch_size, -1, self.num_heads, self.depth)
        return x.permute(0, 2, 1, 3)
  def forward(self, v, k, q, mask=None):
        batch_size = q.size(0)
        # Apply linear transformations to queries, keys, and values
        q = self.wq(q)
        k = self.wk(k)
        v = self.wv(v)

        # Split the heads for multi-head attention
        q = self.split_heads(q, batch_size)
        k = self.split_heads(k, batch_size)
        v = self.split_heads(v, batch_size)

        # Compute scaled dot-product attention
        scaled_attention, attention_weights = scaled_dot_product_attention(q, k, v, mask)
        # Concatenate the attention outputs
        scaled_attention = scaled_attention.permute(0, 2, 1, 3)
        concat_attention = scaled_attention.contiguous().view(batch_size, -1, self.d_model)

        # Pass through the final dense layer
        output = self.dense(concat_attention)

        return output, attention_weights

In [13]:
class EncoderLayer(nn.Module):
  def __init__(self, d_model, num_heads, dff, rate=0.2):
    super(EncoderLayer, self).__init__()
    self.mha = MultiHeadAttention(d_model, num_heads)
    self.ffn = pointwise_ffn(d_model, dff)

    self.layernorm1 = nn.LayerNorm(d_model, eps=1e-6)
    self.layernorm2 = nn.LayerNorm(d_model, eps=1e-6)

    self.dropout1 = nn.Dropout(rate)
    self.dropout2 = nn.Dropout(rate)

  def forward(self, x, training, mask):
    attention_output, _ = self.mha(x, x, x, mask)
    attention_output = self.dropout1(attention_output)
    out1 = self.layernorm1(x + attention_output)

    ffn_output = self.ffn(out1)
    ffn_output = self.dropout2(ffn_output)
    out2 = self.layernorm2(out1 + ffn_output)

    return out2

In [14]:
class DecoderLayer(nn.Module):
  def __init__(self, d_model, num_heads, dff, rate=0.2):
    super(DecoderLayer, self).__init__()

    self.mha1 = MultiHeadAttention(d_model, num_heads)
    self.mha2 = MultiHeadAttention(d_model, num_heads)

    self.fnn = pointwise_ffn(d_model, dff)

    self.layernorm1 = nn.LayerNorm(d_model, eps=1e-6)
    self.layernorm2 = nn.LayerNorm(d_model, eps=1e-6)
    self.layernorm3 = nn.LayerNorm(d_model, eps=1e-6)

    self.dropout1 = nn.Dropout(rate)
    self.dropout2 = nn.Dropout(rate)
    self.dropout3 = nn.Dropout(rate)

  def forward(self, x, enc_output, training, look_ahead_mask, padding_mask):

    att1, att_weights_block1 = self.mha1(x, x, x, look_ahead_mask)
    att1 = self.dropout1(att1)
    out1 = self.layernorm1(att1 + x)

    att2, att_weights_block2 = self.mha2(enc_output, enc_output, out1, padding_mask)
    att2 = self.dropout2(att2)
    out2 = self.layernorm2(att2 + out1)

    ffn_output = self.fnn(out2)
    ffn_output = self.dropout3(ffn_output)
    out3 = self.layernorm3(ffn_output + out2)

    return out3, att_weights_block1, att_weights_block2

In [15]:
class Encoder(nn.Module):
  def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, maximum_position_encoding, rate=0.2):
    super(Encoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.embedding = nn.Embedding(input_vocab_size, d_model)
    # weights_matrix = load_glove_embeddings(input_vocab_size, glove, tokenizer, d_model=300)
    # self.embedding = nn.Embedding.from_pretrained(weights_matrix, freeze=False)

    self.pos_encoding = positional_encoding(maximum_position_encoding, self.d_model)
    self.enc_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)])

    self.dropout = nn.Dropout(rate)

  def forward(self, x, training, mask):
    seq_len = x.shape[1]
    x = self.embedding(x)
    x *= torch.sqrt(torch.tensor(self.d_model, dtype=torch.float32)) #d=d
    x += self.pos_encoding[:, :seq_len, :]
    x = self.dropout(x)
    for i in range(self.num_layers):
      x = self.enc_layers[i](x, training, mask)
    return x

In [16]:
class Decoder(nn.Module):
  def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size, maximum_position_encoding, rate=0.2):
    super(Decoder, self).__init__()
    self.d_model = d_model
    self.num_layers = num_layers

    self.embedding = nn.Embedding(target_vocab_size, d_model)
    # weights_matrix = load_glove_embeddings(target_vocab_size, glove, tokenizer, d_model=300)
    # self.embedding = nn.Embedding.from_pretrained(weights_matrix, freeze=False)


    self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)
    self.dec_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)])
    self.dropout = nn.Dropout(rate)

  def forward(self, x, enc_output, training, look_ahead_mask, padding_mask):

    seq_len = x.shape[1]
    attention_weights = {}
    #GPT

    x = self.embedding(x)
    x *= torch.sqrt(torch.tensor(self.d_model, dtype=torch.float32))
    x += self.pos_encoding[:, :seq_len, :]
    x = self.dropout(x)

    for i in range(self.num_layers):
      x, block1, block2 = self.dec_layers[i](x, enc_output, training, look_ahead_mask, padding_mask)
      attention_weights['decoder_layer{}_block1'.format(i+1)] = block1
      attention_weights['decoder_layer{}_block2'.format(i+1)] = block2
    return x, attention_weights

In [17]:
class Transformer(nn.Module):
  def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, target_vocab_size, pe_input, pe_target, rate=0.2):
    super(Transformer, self).__init__()
    self.encoder = Encoder(num_layers, d_model, num_heads, dff, input_vocab_size, pe_input, rate)

    self.decoder = Decoder(num_layers, d_model, num_heads, dff, target_vocab_size, pe_target, rate)
    self.final_layer = nn.Linear(d_model, target_vocab_size)

  def forward(self, inp, tar, training, enc_padding_mask, look_ahead_mask, dec_padding_mask):

    enc_output = self.encoder(inp, training, enc_padding_mask)

    dec_output, attention_weights = self.decoder(tar, enc_output, training, look_ahead_mask, dec_padding_mask)

    final_output = self.final_layer(dec_output)

    return final_output, attention_weights

In [18]:
transformer = Transformer(
    num_layers=4,
    d_model=300,
    num_heads=6,
    dff=1200,
    input_vocab_size=tokenizer.vocab_size, #32,128
    target_vocab_size=tokenizer.vocab_size, #32,128
    pe_input=512,
    pe_target=512,
).to(device)

In [19]:
checkpoint_path = "/kaggle/input/textsum-with-glove/best_model.pth"
checkpoint = torch.load(checkpoint_path,  map_location=torch.device('cpu'))

transformer.load_state_dict(checkpoint)

print("The checkpoint has been loaded")

/tmp/ipykernel_23/4203467060.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path,  map_location=torch.device('cpu'))


The checkpoint has been loaded


In [20]:
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
epochs = 10
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

loss = nn.CrossEntropyLoss()
optimizer = AdamW(transformer.parameters(), lr=5e-5 ,betas=(0.9, 0.999), eps=1e-8, weight_decay = 0.01)
scheduler = get_linear_schedule_with_warmup(
    optimizer, 
    num_warmup_steps=len(train_dataloader) * 2, 
    num_training_steps=len(train_dataloader) * epochs
)

In [21]:
# def loss_func(real, pred):
#     # Reshape predictions
#     pred = pred.contiguous().view(-1, pred.size(-1))
#     real = real.contiguous().view(-1)
    
#     # Calculate loss
#     loss = F.cross_entropy(
#         pred,
#         real,
#         ignore_index=tokenizer.pad_token_id,
#         reduction='mean'
#     )
#     return loss


# checkpoint_path = '/kaggle/working/checkpoint.pth'
# def save_checkpoint(model, optimizer, epoch, loss, checkpoint_path = '/kaggle/working/checkpoint.pth'):
#     """Save model and optimizer state as a checkpoint."""
#     checkpoint = {
#         # 'epoch': epoch,
#         'model_state_dict': model.state_dict(),
#         # 'optimizer_state_dict': optimizer.state_dict(),
#         # 'loss': loss,
#     }
#     torch.save(checkpoint, checkpoint_path)
#     print(f"Checkpoint saved at epoch {epoch}, loss: {loss:.4f}")
# def create_masks(input_ids, input_mask, target_ids):
#     # Encoder padding mask
#     enc_padding_mask = create_padding_mask(input_ids)
    
#     # Used in the 2nd attention block in the decoder
#     dec_padding_mask = create_padding_mask(input_ids)
    
#     # Used in the 1st attention block in the decoder
#     look_ahead_mask = create_look_ahead_mask(target_ids.size(1)).to(device)
#     dec_target_padding_mask = create_padding_mask(target_ids)
#     combined_mask = torch.max(dec_target_padding_mask, look_ahead_mask)
    
#     return enc_padding_mask, combined_mask, dec_padding_mask

# def train_step(input_ids, input_mask, target_ids):
#     target_input = target_ids[:, :-1]
#     target_real = target_ids[:, 1:]
    
#     enc_padding_mask, combined_mask, dec_padding_mask = create_masks(input_ids, input_mask, target_input)
    
#     optimizer.zero_grad()
#     predictions, _ = transformer(input_ids, target_input, True, 
#                                enc_padding_mask, combined_mask, dec_padding_mask)
    
#     loss = loss_func(target_real, predictions)
#     loss.backward()
#     torch.nn.utils.clip_grad_norm_(transformer.parameters(), max_norm=1.0)
#     optimizer.step()
#     scheduler.step()
    
#     return loss.item()

# def evaluate(val_dataset, batch_size=64):
#     transformer.eval()
#     total_loss = 0
#     val_dataloader = DataLoader(val_dataset, batch_size=batch_size)
    
#     with torch.no_grad():
#         for batch in val_dataloader:
#             input_ids, input_mask, target_ids = [x.to(device) for x in batch]
#             target_input = target_ids[:, :-1]
#             target_real = target_ids[:, 1:]
            
#             enc_padding_mask, combined_mask, dec_padding_mask = create_masks(input_ids, input_mask, target_input)
            
#             predictions, _ = transformer(input_ids, target_input, False,
#                                       enc_padding_mask, combined_mask, dec_padding_mask)
            
#             loss = loss_func(target_real, predictions)
#             total_loss += loss.item()
    
#     return total_loss / len(val_dataloader)

# # Training loop
# batch_size = 64
# epochs = 10
# train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# best_val_loss = float('inf')
# patience = 4
# trigger_times = 0

# for epoch in range(epochs):
#     transformer.train()
#     total_loss = 0
#     batch_count = 0
    
#     print(f'\nEpoch {epoch + 1}/{epochs}')
#     print('-' * 40)
    
#     for batch_idx, batch in enumerate(train_dataloader):
#         input_ids, input_mask, target_ids = [x.to(device) for x in batch]
#         batch_loss = train_step(input_ids, input_mask, target_ids)
#         total_loss += batch_loss
#         batch_count += 1
#         # Print progress every 100 batches
#         if batch_idx % 100 == 0:
#             print(f'Batch {batch_idx}/{len(train_dataloader)}, Loss: {batch_loss:.4f}')
    
#     # Calculate average loss for the epoch
#     avg_train_loss = total_loss / batch_count
    
#     # Validation at the end of each epoch
#     val_loss = evaluate(val_dataset)
#     if val_loss < best_val_loss:
#         best_val_loss = val_loss
#         trigger_times = 0
#         torch.save(transformer.state_dict(), 'best_model.pth')
#         print("save there")
#     else:
#         trigger_times += 1
#         if trigger_times >= patience:
#             print("Early stopping triggered")
#             break
#     # Save checkpoint
#     save_checkpoint(transformer, optimizer, epoch + 1, avg_train_loss)    
#     # Print epoch summary
#     print('\nEpoch Summary:')
#     print(f'Average Training Loss: {avg_train_loss:.4f}')
#     print(f'Validation Loss: {val_loss:.4f}')
#     print(f'Learning Rate: {scheduler.get_last_lr()[0]:.2e}')

In [22]:
def count_params(model, is_human: bool = False):
    params: int = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return f"{params / 1e6:.2f}M" if is_human else params

print(transformer)
print("Total # of params:", count_params(transformer, is_human=True))

Transformer(
  (encoder): Encoder(
    (embedding): Embedding(32000, 300)
    (enc_layers): ModuleList(
      (0-3): 4 x EncoderLayer(
        (mha): MultiHeadAttention(
          (wq): Linear(in_features=300, out_features=300, bias=True)
          (wk): Linear(in_features=300, out_features=300, bias=True)
          (wv): Linear(in_features=300, out_features=300, bias=True)
          (dense): Linear(in_features=300, out_features=300, bias=True)
        )
        (ffn): Sequential(
          (0): Linear(in_features=300, out_features=1200, bias=True)
          (1): ReLU()
          (2): Linear(in_features=1200, out_features=300, bias=True)
        )
        (layernorm1): LayerNorm((300,), eps=1e-06, elementwise_affine=True)
        (layernorm2): LayerNorm((300,), eps=1e-06, elementwise_affine=True)
        (dropout1): Dropout(p=0.2, inplace=False)
        (dropout2): Dropout(p=0.2, inplace=False)
      )
    )
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (decoder): Decoder(
    (em

In [23]:
def generate_summary(model, tokenizer, article, max_length=50, device=device):
    """
    Generate a summary for a given article using the trained transformer model.
    
    Args:
        model: Trained transformer model
        tokenizer: T5Tokenizer instance
        article: Input article text (string)
        max_length: Maximum length of generated summary
        device: Computation device (cuda/cpu)
    
    Returns:
        str: Generated summary
    """
    model.eval()
    
    # Preprocess and tokenize the input article
    article = article.lower()
    article = re.sub(r'[^a-z0-9\s.,!?%$&@#:]', '', article)
    article = ' '.join(article.split())
    
    # Encode the article
    input_ids = tokenizer(
        article,
        max_length=500,
        padding='max_length',
        truncation=True,
        return_tensors="pt"
    )
    
    # Move input to device
    input_ids = input_ids['input_ids'].to(device)
    attention_mask = (input_ids != 0).float().to(device)
    
    # Initialize target with start token
    decoder_input = torch.tensor([[tokenizer.pad_token_id]]).to(device)
    
    with torch.no_grad():
        for _ in range(max_length):
            # Create masks
            enc_padding_mask = create_padding_mask(input_ids)
            combined_mask = create_look_ahead_mask(decoder_input.size(1)).to(device)
            dec_padding_mask = create_padding_mask(input_ids)
            
            # Get predictions
            predictions, _ = model(
                input_ids,
                decoder_input,
                False,
                enc_padding_mask,
                combined_mask,
                dec_padding_mask
            )
            
            # Get the next token prediction
            next_token = predictions[:, -1:].argmax(dim=-1)
            decoder_input = torch.cat([decoder_input, next_token], dim=-1)
            
            # Stop if end token is generated
            if next_token.item() == tokenizer.eos_token_id:
                break
    
    # Decode the generated tokens
    generated_ids = decoder_input[0].cpu().numpy()
    generated_text = tokenizer.decode(generated_ids, skip_special_tokens=True)
    
    return generated_text

# Example usage:
def test_summary_generation():
    # Load your best model checkpoint
    # checkpoint = torch.load('/kaggle/working/transformer_best_model.pth')
    # transformer.load_state_dict(checkpoint['model_state_dict'])
    
    # Test article
    test2 = '''
    Peter Whittle said he would back Paul Nuttall as "a leader who knows the party inside out and who can command the loyalty" of all members. Former deputy leader Mr Nuttall, former deputy chairwoman Suzanne Evans and ex- soldier John Rees-Evans remain in the contest, to be decided on 28 November. Raheem Kassam also withdrew from the contest on Monday. In a message on Facebook, Mr Whittle said Thursday's High Court ruling, which found that the government could not trigger Article 50 - the formal process of leaving the EU - without consulting Parliament, showed that "the battle to ensure that the Leave vote in the referendum is respected is far from over and UKIP is needed more than ever". Mr Whittle, the party's culture spokesman and a former London mayoral candidate, added that the "sheer breadth of Paul's political experience, his dedication to the values of the party and the obvious affection in which he is held by members make him the person who is best placed to take us forward". Nigel Farage is back as interim leader after his successor Diane James quit just 18 days into the job. Mr Kassam, a former aide to Mr Farage, had endorsed Mr Whittle after himself pulling out of the race.
    '''
    test_article = '''
     
Gundogan, 26, told BBC Sport he "can see the finishing line" after tearing cruciate knee ligaments in December, but will not rush his return. The German missed the 2014 World Cup following back surgery that kept him out for a year, and sat out Euro 2016 because of a dislocated kneecap. He said: "It is heavy mentally to accept that." Gundogan will not be fit for the start of the Premier League season at Brighton on 12 August but said his recovery time is now being measured in "weeks" rather than months. He told BBC Sport: "It is really hard always to fall and fight your way back. You feel good and feel ready, then you get the next kick. "The worst part is behind me now. I want to feel ready when I am fully back. I want to feel safe and confident. I don't mind if it is two weeks or six." Gundogan made 15 appearances and scored five goals in his debut season for City following his £20m move from Borussia Dortmund. He is eager to get on the field again and was impressed at the club's 4-1 win over Real Madrid in a pre-season game in Los Angeles on Wednesday. Manager Pep Guardiola has made five new signings already this summer and continues to have an interest in Arsenal forward Alexis Sanchez and Monaco's Kylian Mbappe. Gundogan said: "Optimism for the season is big. It is huge, definitely. "We felt that last year as well but it was a completely new experience for all of us. We know the Premier League a bit more now and can't wait for the season to start." City complete their three-match tour of the United States against Tottenham in Nashville on Saturday. Chelsea manager Antonio Conte said earlier this week he did not feel Tottenham were judged by the same standards as his own side, City and Manchester United. Spurs have had the advantage in their recent meetings with City, winning three and drawing one of their last four Premier League games. And Gundogan thinks they are a major threat. He said: "Tottenham are a great team. They have the style of football. They have young English players. Our experience last season shows it is really tough to beat them. "They are really uncomfortable to play against. "I am pretty sure, even if they will not say it loud, the people who know the Premier League know Tottenham are definitely a competitor for the title."    
    '''
    
    # Generate summary
    summary = generate_summary(transformer, tokenizer, test_article)
    
    print("Original Article:")
    print(test_article)
    print("\nGenerated Summary:")
    print(summary)
    summary1 = generate_summary(transformer, tokenizer, test2)
    
    print("Original Article:")
    print(test2)
    print("\nGenerated Summary:")
    print(summary1)
# Run the test
test_summary_generation()

# You can also use it directly:
# article = "Your article text here..."
# summary = generate_summary(transformer, tokenizer, article)
# print(summary)

Original Article:

     
Gundogan, 26, told BBC Sport he "can see the finishing line" after tearing cruciate knee ligaments in December, but will not rush his return. The German missed the 2014 World Cup following back surgery that kept him out for a year, and sat out Euro 2016 because of a dislocated kneecap. He said: "It is heavy mentally to accept that." Gundogan will not be fit for the start of the Premier League season at Brighton on 12 August but said his recovery time is now being measured in "weeks" rather than months. He told BBC Sport: "It is really hard always to fall and fight your way back. You feel good and feel ready, then you get the next kick. "The worst part is behind me now. I want to feel ready when I am fully back. I want to feel safe and confident. I don't mind if it is two weeks or six." Gundogan made 15 appearances and scored five goals in his debut season for City following his £20m move from Borussia Dortmund. He is eager to get on the field again and was impr

In [24]:
def generate_summary(model, tokenizer, article, max_length=50, device=device):
    """
    Generate a summary for a given article using the trained transformer model.
    
    Args:
        model: Trained transformer model
        tokenizer: T5Tokenizer instance
        article: Input article text (string)
        max_length: Maximum length of generated summary
        device: Computation device (cuda/cpu)
    
    Returns:
        str: Generated summary
    """
    model.eval()
    
    # Preprocess and tokenize the input article
    article = article.lower()
    article = re.sub(r'[^a-z0-9\s.,!?%$&@#:]', '', article)
    article = ' '.join(article.split())
    
    # Encode the article
    input_ids = tokenizer(
        article,
        max_length=512,
        padding='max_length',
        truncation=True,
        return_tensors="pt"
    )
    
    # Move input to device
    input_ids = input_ids['input_ids'].to(device)
    attention_mask = (input_ids != tokenizer.pad_token_id).float().to(device)
    
    # Initialize target with start token
    start_token = tokenizer.encode('<s>', add_special_tokens=False)
    if not start_token:
        start_token = [tokenizer.bos_token_id if tokenizer.bos_token_id is not None else tokenizer.cls_token_id]
    decoder_input = torch.tensor([start_token], dtype=torch.long).to(device)
    
    with torch.no_grad():
        for _ in range(max_length):
            # Create masks
            enc_padding_mask = create_padding_mask(input_ids)
            look_ahead_mask = create_look_ahead_mask(decoder_input.size(1)).to(device)
            dec_padding_mask = create_padding_mask(input_ids)
            
            # Get predictions
            predictions, _ = model(
                input_ids,
                decoder_input,
                False,
                enc_padding_mask,
                look_ahead_mask,
                dec_padding_mask
            )
            
            # Get the next token prediction
            next_token = predictions[:, -1:].argmax(dim=-1)
            decoder_input = torch.cat([decoder_input, next_token], dim=-1)
            
            # Stop if end token is generated
            if next_token.item() in [tokenizer.eos_token_id, tokenizer.sep_token_id]:
                break
    
    # Decode the generated tokens
    generated_ids = decoder_input[0].cpu().numpy()
    generated_text = tokenizer.decode(generated_ids, skip_special_tokens=True)
    
    return generated_text

In [25]:
def test_summary_generation():
    # Load your best model checkpoint
    # checkpoint_path = '/kaggle/working/best_checkpoint.pth'
    # checkpoint = torch.load(checkpoint_path, map_location=device)
    # transformer.load_state_dict(checkpoint['model_state_dict'])
    
    # Set model to evaluation mode
    transformer.eval()
    
    # Sample articles for testing
    test_articles = [
        """
        Scientists have made a groundbreaking discovery in the field of artificial intelligence. 
        The new research, published in Nature, demonstrates how neural networks can now learn 
        from significantly less data while maintaining high accuracy. This breakthrough could 
        revolutionize how AI systems are trained, making them more efficient and accessible 
        to researchers with limited computational resources. The team's approach combines 
        traditional deep learning methods with novel algorithmic techniques.
        """,
        '''
Gundogan, 26, told BBC Sport he "can see the finishing line" after tearing cruciate knee ligaments in December, but will not rush his return. The German missed the 2014 World Cup following back surgery that kept him out for a year, and sat out Euro 2016 because of a dislocated kneecap. He said: "It is heavy mentally to accept that." Gundogan will not be fit for the start of the Premier League season at Brighton on 12 August but said his recovery time is now being measured in "weeks" rather than months. He told BBC Sport: "It is really hard always to fall and fight your way back. You feel good and feel ready, then you get the next kick. "The worst part is behind me now. I want to feel ready when I am fully back. I want to feel safe and confident. I don't mind if it is two weeks or six." Gundogan made 15 appearances and scored five goals in his debut season for City following his £20m move from Borussia Dortmund. He is eager to get on the field again and was impressed at the club's 4-1 win over Real Madrid in a pre-season game in Los Angeles on Wednesday. Manager Pep Guardiola has made five new signings already this summer and continues to have an interest in Arsenal forward Alexis Sanchez and Monaco's Kylian Mbappe. Gundogan said: "Optimism for the season is big. It is huge, definitely. "We felt that last year as well but it was a completely new experience for all of us. We know the Premier League a bit more now and can't wait for the season to start." City complete their three-match tour of the United States against Tottenham in Nashville on Saturday. Chelsea manager Antonio Conte said earlier this week he did not feel Tottenham were judged by the same standards as his own side, City and Manchester United. Spurs have had the advantage in their recent meetings with City, winning three and drawing one of their last four Premier League games. And Gundogan thinks they are a major threat. He said: "Tottenham are a great team. They have the style of football. They have young English players. Our experience last season shows it is really tough to beat them. "They are really uncomfortable to play against. "I am pretty sure, even if they will not say it loud, the people who know the Premier League know Tottenham are definitely a competitor for the title."        '''
    ]
    
    # Generate and print summaries
    for idx, article in enumerate(test_articles):
        summary = generate_summary(transformer, tokenizer, article)
        print(f"\nOriginal Article {idx + 1}:\n{article}")
        print(f"\nGenerated Summary {idx + 1}:\n{summary}\n")
test_summary_generation()


Original Article 1:

        Scientists have made a groundbreaking discovery in the field of artificial intelligence. 
        The new research, published in Nature, demonstrates how neural networks can now learn 
        from significantly less data while maintaining high accuracy. This breakthrough could 
        revolutionize how AI systems are trained, making them more efficient and accessible 
        to researchers with limited computational resources. The team's approach combines 
        traditional deep learning methods with novel algorithmic techniques.
        

Generated Summary 1:
s> has been mapped by scientists in a new study of how to manage our brains.


Original Article 2:

Gundogan, 26, told BBC Sport he "can see the finishing line" after tearing cruciate knee ligaments in December, but will not rush his return. The German missed the 2014 World Cup following back surgery that kept him out for a year, and sat out Euro 2016 because of a dislocated kneecap. He said: "I

In [26]:
def test_summary_generation_from_data():
    """
    Test summary generation on sample articles from the training dataset
    """
    # Get 3 random samples from training data
    import random
    sample_indices = random.sample(range(len(X_traindata)), 3)

    for idx in sample_indices:
        # Get article and its actual summary
        article = X_traindata.iloc[idx]
        actual_summary = y_traindata.iloc[idx]

        # Generate summary using the model
        generated_summary = generate_summary(transformer, tokenizer, article)

        # Print results
        print("\n" + "="*80)
        print(f"Example {sample_indices.index(idx) + 1}")
        print("\nORIGINAL ARTICLE:")
        print(article)
        print("\nACTUAL SUMMARY:")
        print(actual_summary)
        print("\nGENERATED SUMMARY:")
        print(generated_summary)
        print("="*80)

# Run the test
test_summary_generation_from_data()


Example 1

ORIGINAL ARTICLE:
After a slow start from United, Daley Blind set up Robin van Persie to break the deadlock with a precise finish.
Radamel Falcao followed up Angel Di Maria's shot to make it 2-0 before Wes Morgan headed into his own net to cap a miserable first half for the Foxes.
Leicester belatedly replied through Marcin Wasilewski's late header.
Media playback is not supported on this device
Louis van Gaal's side were not at their best but they did not have to be against a Foxes side who offered little in attack.
United let slip a 2-0 lead and were beaten 5-3 when these sides met at King Power Stadium in September, but there was never a hint of a similar fightback from the Foxes this time.
Leicester concentrated on frustrating their hosts in the early stages and that game-plan was working until Andrej Kramaric allowed Blind to steal possession after a United attack had broken down.
Blind looked up and released Van Persie, who was given the benefit of the doubt by the lin